<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 7</h1>
</div>


# Enums, Structs, and Introduction to Memory Management

## Enums

* Data type allowing for capturing a small set of options


In [ ]:
enum Direction {
    North,
    East,
    South,
    West,
    SouthWest,
}

let dir = Direction::North;
let dir_2: Direction = Direction::South;
let dir_3: Direction = Direction::SouthWest;

In [ ]:
// Avoiding specifying "Direction::"
use Direction::East;
let dir_3 = East;

In [ ]:
// Bringing two options into the current scope
use Direction::{East,West};
let dir_3 = West;

In [ ]:
// Bringing all options in
use Direction::*;
let dir_4 = South;

## Enums: pattern matching via `match`

In [ ]:
enum Direction {
    North,
    East,
    South,
    West,
}
let dir = Direction::North;

// print the direction
match dir {
    // if things not in scope,
    // have to use "Direction::" 
    Direction::North => println!("N"),
    // but they are, so we don't have to
    South => println!("S"),
    West => println!("W"),
    East => println!("E"),
};

In [ ]:
let dir_2: Direction = Direction::South;

// won't work 
match dir_2 {
    North => println!("N"),
    South => println!("S"),
    // East and West not covered
};

In [ ]:
let dir_2: Direction = Direction::South;

match dir_2 {
    North => println!("N"),
    South => println!("S"),
    
    // match anything left
    _ => (),
}

In [ ]:
match dir_2 {
    _ => (),
    
    // will never get here!!
    North => println!("N"),
    South => println!("S"),
}

### What is match?
* Type of a switch statement in C/C++ (Python doesn't have an equivalent)
* Must be exhaustive though there is a way to specify default (_ =>) 

## Enums

* Each option can come with additional information

In [ ]:
#[derive(Debug)]
enum DivisionResult {
    Ok(u32),
    DivisionByZero,
}

fn divide(x:u32, y:u32) -> DivisionResult {
    if y == 0 {
        return DivisionResult::DivisionByZero;
    } else {
        return DivisionResult::Ok(x / y);
    }
}

let (a,b) = (9,0);
match divide(a,b) {
    DivisionResult::Ok(result)
        => println!("the result is {}",result),
    DivisionResult::DivisionByZero
        => println!("noooooo!!!!"),
};

let z = divide(5, 4);
println!("The result is {:?}", z);

In [ ]:
enum DivisionResult {
    Ok(u32,u32),
    DivisionByZero,
}

fn divide(x:u32, y:u32) -> DivisionResult {
    if y == 0 {
        DivisionResult::DivisionByZero
    } else {
        DivisionResult::Ok(x / y, x % y)
    }
}

let (a,b) = (9,4);
match divide(a,b) {
    DivisionResult::Ok(result,reminder) => {
            println!("the result is {}",result);
            println!("the reminder is {}",reminder);
    }
    DivisionResult::DivisionByZero
        => println!("noooooo!!!!"),
};

## Displaying enums
By default Rust doesn't know how to display it

In [ ]:
let dir = Direction::North;
println!("{:?}",dir);

In [ ]:
let dir = Direction::North;


dir

In [ ]:
#[derive(Debug)]
enum Direction {
    North,
    East,
    South,
    West,
}

use Direction::*;
let dir = Direction::North;
println!("{:?}",dir);

### What is #[derive(Debug)]?

* A simple way to tell Rust to generate code that allows a complex type to be printed
* But you can also do it manually!!!

```
use std::fmt;

impl fmt::Debug for Direction {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
           match *self {
               Direction::North => write!(f, "North"),
               Direction::East => write!(f, "East"),
               Direction::South => write!(f, "South"),
               Direction::West => write!(f, "West"),               
           }
    }
}
```

In [ ]:
let dir = Direction::North;
dir

In [ ]:
let dir = Direction::North;
println!("{:?}",dir);

In [ ]:
// Example of how make a complex datatype printable directly (without deriving from Debug)
use std::fmt;

impl fmt::Display for Direction {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
           match *self {
               Direction::North => write!(f, "North"),
               Direction::East => write!(f, "East"),
               Direction::South => write!(f, "South"),
               Direction::West => write!(f, "West"),               
           }
    }
}
println!("{}", dir);

## `match` as expression

In [ ]:
// swap east and west
let mut dir_4 = West;
println!("{:?}", dir_4);

dir_4 = match dir_4 {
    East => West,
    West => {
        println!("Switching West to East");
        East
    }
    // variable mathching anything else
    _ => West,

};

println!("{:?}", dir_4);

## Simplified matching `if let`

Consider the following example (in which we want to use just one branch):

In [ ]:
#[derive(Debug)]
enum DivisionResult {
    Ok(u32,u32),
    DivisionByZero,
}

fn divide(x:u32, y:u32) -> DivisionResult {
    if y == 0 {
        DivisionResult::DivisionByZero
    } else {
        DivisionResult::Ok(x / y, x % y)
    }
}


match divide(8,3) {
    DivisionResult::Ok(result,remainder) => println!("{} (remainder {})",result,remainder),
    _ => (), // <--- how to avoid this?
};


`if let` allows for matching just one branch

In [ ]:
if let DivisionResult::Ok(result,reminder) = divide(8,7) { 
    println!("{} (remainder {})",result,reminder);
};

In [ ]:
use Direction::*;
let dir = North;
if let North = dir {
    println!("North");
};

In [ ]:
// But it is important to have the enum
// on the left hand side
if let West = dir {
// if let dir = West {
 println!("North");
} else {
    println!("Something else");
};

In [ ]:
// Don't do this.
if dir == North {
    println!("North");
}

## Structs

Last time: tuples, e.g., `(12, 1.7, true)`

Structs compared to tuples:<br>
$\bullet$ **Similar:** can hold a few items of different types<br>
$\bullet$ **Different:** the items have names

In [ ]:
// Definition: list items (called fields)
//             and their types

struct Person {
    name: String,
    year_born: u16,
    time_100m: f64,
    likes_ice_cream: bool,
}

In [ ]:
// Instantiation: replace types with values

let mut cartoon_character: Person = Person {
    name: String::from("Tasmanian Devil"),
    year_born: 1954,
    time_100m: 7.52,
    likes_ice_cream: true,
};


In [ ]:
// Accessing fields: use ".field_name"
println!("{} was born in {}",
    cartoon_character.name,
    cartoon_character.year_born);
cartoon_character.year_born = 2022;
println!("{} was born in {}",
    cartoon_character.name,
    cartoon_character.year_born);

<br>
<div align="center">
    <b>Structs vs tuples:<br>Which are better?</b>
</div>


## Tuple structs

Named tuples to impose more meaning and delineate a different type.  Notice the different syntax () vs {}.

Example: both `(f64,f64,f64)`
* box size (e.g., 8.5 in${}\times{}$11 in${}\times{}$6 in)
* Euclidean coordinates of a point in 3D

In [ ]:
struct BoxSize(f64,f64,f64);
struct Point2(f64,f64,f64);

In [ ]:
let mut my_box = BoxSize(3.2,6.0,2.0);
let mut p : Point2 = Point2(-1.3,2.1,0.0);

In [ ]:
// won't work
my_box = p;

// Impossible to accidentally confuse different
// types of triples.
// No runtime penalty! Verified at compilation.

In [ ]:
// Acessing via index
println!("{} {} {}",p.0,p.1,p.2);
p.0 = 17.2;

// Destructuring
let Point2(first,second,third) = p;
println!("{} {} {}", first, second, third);

## Named structs in enums

Structs with braces and exchangable with tuples in many places

In [ ]:
enum LPSolution {
    None,
    Point{x:f64,y:f64}
}

let example = LPSolution::Point{x:1.2, y:4.2};

In [ ]:
if let LPSolution::Point{x:first,y:second} = example {
    println!("coordinates: {} {}", first, second);
};

### Structs are critical to Rust's OO capabilities

In [ ]:
struct Point {
    x: f64,
    y: f64,
}

struct Rectangle {
    p1: Point,
    p2: Point,
}

impl Rectangle {
    // This is a method
    fn area(&self) -> f64 {
        // `self` gives access to the struct fields via the dot operator
        let Point { x: x1, y: y1 } = self.p1;
        let Point { x: x2, y: y2 } = self.p2;

        // `abs` is a `f64` method that returns the absolute value of the
        // caller
        ((x1 - x2) * (y1 - y2)).abs()
    }

    fn perimeter(&self) -> f64 {
        let Point { x: x1, y: y1 } = self.p1;
        let Point { x: x2, y: y2 } = self.p2;

        2.0 * ((x1 - x2).abs() + (y1 - y2).abs())
    }
}

let rectangle = Rectangle {
    p1: Point{x:0.0, y:0.0},
    p2: Point{x:3.0, y:4.0},
};

println!("Rectangle perimeter: {}", rectangle.perimeter());
println!("Rectangle area: {}", rectangle.area());


## Memory Management: Stack vs. Heap

* Two different places where space for data can be allocated
* We will discuss them one by one


## Stack

* LIFO (last in first out) memory allocation
* Stores current local variables and additional information such as:
  - function arguments
  - function output
  - where to continue when a function terminates
* Fast memory allocation
* Usually small fraction of the memory
* Often: size of the allocated memory has to be known in advance (compilation time)

Almost everything you saw so far allocated on stack
* Exception: data in `String` allocated on heap

## Stack example (idealized)

In [ ]:
fn main() {
    let mut x = 3;
    let mut y = 8;
    println!("x = {}, y = {}",x,y);
    x = add_or_subtract(x,y,true); // x = x + y
    y = add_or_subtract(x,y,false); // y = x - y
    x = add_or_subtract(x,y,false); // x = x - y
    println!("x = {}, y = {}",x,y);
}

fn add_or_subtract(x:i32, y:i32, add:bool) -> i32 {
    let second_arg = if add {y} else {negate(y)};
    x + second_arg
}

fn negate(x:i32) -> i32 {
    -x
}

main();


### Step 1: call `main`
* `x` and `y` allocated on stack and initiated
* Stack: `main` (`x`, `y`)

### Step 2: call `add_or_subtract` (1st time)
* arguments for `add_or_subtract` put on stack
* space for solution allocated on stack
* space for `second_arg` allocated as well
* Stack: `main` (`x`, `y`), `add_or_subtract` (all the above + auxiliary information)

## Stack example (idealized)

In [ ]:
fn main() {
    let mut x = 3;
    let mut y = 8;
    println!("x = {}, y = {}",x,y);
    x = add_or_subtract(x,y,true);
    y = add_or_subtract(x,y,false);
    x = add_or_subtract(x,y,false);
    println!("x = {}, y = {}",x,y);
}

fn add_or_subtract(x:i32, y:i32, add:bool) -> i32 {
    let second_arg = if add {y} else {negate(y)};
    x + second_arg
}

fn negate(x:i32) -> i32 {
    -x
}

main();


### Step 3: `add_or_subtract` terminates
* process and remove all information about the 
call
* Stack: `main` (`x`, `y`)

### Step 4: call `add_or_subtract` (2nd time)
* arguments for `add_or_subtract` put on stack
* space for solution allocated on stack
* space for `second_arg` allocated as well
* Stack: `main` (`x`, `y`), `add_or_subtract` (all the above + auxiliary information)

## Stack example (idealized)

In [ ]:
fn main() {
    let mut x = 3;
    let mut y = 8;
    println!("x = {}, y = {}",x,y);
    x = add_or_subtract(x,y,true);
    y = add_or_subtract(x,y,false);
    x = add_or_subtract(x,y,false);
    println!("x = {}, y = {}",x,y);
}

fn add_or_subtract(x:i32, y:i32, add:bool) -> i32 {
    let second_arg = if add {y} else {negate(y)};
    x + second_arg
}

fn negate(x:i32) -> i32 {
    -x
}

main();


### Step 5: call `negate` (1st time)
* the argument for `negate` put on stack
* space for solution allocated on stack
* Stack: `main` (`x`, `y`), `add_or_subtract` (...), `negate` (all of the above + auxiliary information)

### Step 6: `negate` terminates
* process and remove all information about the 
call
* Stack: `main` (`x`, `y`), `add_or_subtract` (...)

## Stack example (idealized)

In [ ]:
fn main() {
    let mut x = 3;
    let mut y = 8;
    println!("x = {}, y = {}",x,y);
    x = add_or_subtract(x,y,true);
    y = add_or_subtract(x,y,false);
    x = add_or_subtract(x,y,false);
    println!("x = {}, y = {}",x,y);
}

fn add_or_subtract(x:i32, y:i32, add:bool) -> i32 {
    let second_arg = if add {y} else {negate(y)};
    x + second_arg
}

fn negate(x:i32) -> i32 {
    -x
}

main();



### Step 7: `add_or_subtract` terminates
* [...]
* Stack: `main` (`x`, `y`)

### Step 8: call `add_or_subtract` (3rd time)
* [...]
* Stack: `main` (`x`, `y`), `add_or_subtract` (...)

<div align="center">
    <h1>...</h1>
</div>

## Limited space on stack!

In [ ]:

fn same_number(x:u32) -> u32 {
    match x {
        0 => 0,
        _ => 1 + same_number(x - 1),
    }
}

In [ ]:
same_number(7)

In [ ]:
same_number(123_456)

In [ ]:
same_number(1_000_000)

## Using too much memory on stack: *stack overflow*

This is where the name of the popular webpage for asking questions about programming comes from!<br>

<div align="center">
    <img src="chucknorris.png" alt="[screenshot of stackoverflow.com]">
</div>

## Heap

* Memory allocated and freed in arbitrary order
* Arbitrary amount allocated
* The application knows a *pointer*${}={}$the address of assigned memory

<div align="center">
<b>Pros and cons?</b>
</div>

Pros:
* Arbitrary amount of data
* No copying to pass data around
  * Just share the pointer!


Cons:
* Slower allocation:
  * Possible request for more space to the operating system
* Possible memory fragmentation
* Slower access:
  * Have to follow the pointer to get to data


## Stack vs. heap in Python

* Elementary pieces of data allocated on stack: integers, floats, Boolean values, ...

* Anything else allocated on the heap

## Heap management

**Memory allocation:**
* ask for a given amount of space 
* receives a pointer to it<br> (or an out of memory error)

**Freeing memory:**
* classical manual: explicitly return it
  * more complicated
* automatic: garbage collection
  * comes with additional costs



C: `malloc` / `free`

C++: `new` / `delete` + C

**Pitfalls of manual memory management:**

 * leaks: unused memory never returned

 * attempting to use a pointer to memory that was deallocated

 * returning memory that was already deallocated

<div align="center">
    <b>How does Rust deal with these problems?</b>
</div>

## Allocating on the heap in Rust

* Various methods. The simplest via `Box::new(...)`
* Also `Vec, Rc, Arc, Mutex, Cell, RefCell`, and many more in `std::collections::*`

In [ ]:
// placing integers on the heap
let mut pointer = Box::new(2000_i32);
let pointer2 : Box<i32> = Box::new(22);

In [ ]:
// accessing data via a * operator
println!("vals: {} {}", *pointer, *pointer2);
println!("vals: {} {}", pointer, pointer2);
println!("vals: {} {}", &pointer, &pointer2);
println!("sum: {}", *pointer + *pointer2);
// println!("vals: {:p} {:p}", pointer, pointer2);
// println!("sum: {}", pointer + pointer2);
// println!("sum: {}", &pointer + &pointer2);

In [ ]:
*pointer = *pointer2;
println!("vals: {} {}", pointer, pointer2);
println!("sum: {}", *pointer + *pointer2);

In [ ]:
*pointer2 = 23;
println!("sum: {}", *pointer + *pointer2);